<a href="https://colab.research.google.com/github/cyan9212/Machine_Learning_Basics/blob/main/GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import uniform, randint


#데이터 불러오기
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.info()
wine.describe()
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

#훈련 세트/ 검증 세트/ 테스트 세트
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(data.shape, sub_input.shape, val_input.shape, test_input.shape, end='\n')

#DecisionTree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print('Train_set: ', dt.score(sub_input, sub_target))
print('Val_set: ', dt.score(val_input, val_target))
print()

#5폴드 교차검증
scores = cross_validate(dt, train_input,  train_target)
print(scores)
print(np.mean(scores['test_score']))
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

#10폴드 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) #Train_set을 섞기 위해 splitter를 사용
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

#그리드서치
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} #탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print('Best_Estimator: ', dt.score(train_input, train_target))
print('Best_params: ', gs.best_params_)
print(gs.cv_results_['mean_test_score'])
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), 'max_depth':range(5,20,1), 'min_samples_split':range(2,100,10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

#randint, uniform test
rgen = randint(0,10)
rgen.rvs(10)
np.unique(rgen.rvs(1000), return_counts=True)

ugen = uniform(0,1)
ugen.rvs(10)

#랜덤서치
params = {'min_impurity_decrease':uniform(0.0001,0.001), 'max_depth':randint(20,50), 'min_samples_split':randint(2,25), 'min_samples_leaf':randint(1,25)}
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB
(6497, 3) (4157, 3) (1040, 3) (1300, 3)
Train_set:  0.9971133028626413
Val_set:  0.864423076923077

{'fit_time': array([0.00661469, 0.00707126, 0.0073266 , 0.00689626, 0.00652647]), 'score_time': array([0.00070691, 0.00089645, 0.00076222, 0.0007782 , 0.00069046]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
0.855300214703487
0.8574181117533719
Best_Estimator:  0.9615162593804117
Best_params:  {'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}
{'max_depth': 14, 'min_impurity_decrease': 0.0